## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [ ]:
from dataclasses import dataclass
from re import X
from typing import Union, Callable
from numbers import Number
import numpy as np
from scipy.misc import derivative
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        try:
            match other:
                case Dual(o_value, o_d):
                    return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / o_value**2)
                case Number():
                    return Dual(self.value / float(other), self.d / float(other))
        except ZeroDivisionError as error:
            print(f"Can't divide by zero! {error=}")
            exit(-1)

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                # the implementation of the log() function is in the cell below
                return Dual(self.value ** o_value, self.value ** o_value * (other * log(self)).d)
            case Number():
                return Dual(self.value ** float(other), float(other) * self.value ** (other - 1))
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.value), self.d * np.sign(self.value))

    __rmul__ = __mul__
    __radd__ = __add__

    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, self.value ** o_value * (other * log(self)).d)
            case Number():
                return Dual(float(other) ** self.value, float(other) ** self.value * log(float(other))) 

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        try:
            match other:
                case Dual(o_value, o_d):
                    return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / o_value ** 2)
                case Number():
                    return Dual(float(other) / self.value, - self.d * float(other) / self.value ** 2)
        except ZeroDivisionError as error:
            print(f"Can't divide by zero! {error=}")
            exit(-1)

    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, - self.d)


def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    def newfunc(*args):
        answers = []
        for j in range(0, len(args)):
            dual_args = [Dual(args[i], float(i==j)) for i in range(0, len(args))]
            ans = func(*dual_args)
            if type(ans) != Dual:
                ans = 0
            else:
                ans = ans.d
            answers.append(ans)
        return answers
    return newfunc    

# test functions
f1 = lambda x: 5 * x * x + 2 * x + 2
f2 = lambda x: x**2 - 3 * x ** (-5)
f3 = lambda x: - 3
f4 = lambda x: - x - x / (2 * x + 1)
f5 = lambda x: abs(x - 1)
f6 = lambda x: 35 - x + 245 / x

f1_diff = diff(f1)
f2_diff = diff(f2)
f3_diff = diff(f3)
f4_diff = diff(f4)
f5_diff = diff(f5)
f6_diff = diff(f6)

# results
print(f1_diff(3.)[0] - derivative(f1, 3., dx=1e-6))
print(f2_diff(-1.)[0] - derivative(f2, -1., dx=1e-6))
print(f3_diff(4.)[0] - derivative(f3, 4., dx=1e-6))
print(f4_diff(2.53)[0] - derivative(f4, 2.53, dx=1e-6))
print(f5_diff(1.)[0] - derivative(f5, 1., dx=1e-6))
print(f6_diff(1000.)[0] - derivative(f6, 1000., dx=1e-6))


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [ ]:
def exp(arg: Union["Dual", Number]) -> "Dual":
    match arg:
        case Dual(arg.value, arg.d):
            return Dual(math.exp(arg.value), math.exp(arg.value) * arg.d)
        case Number():
            return math.exp(arg)

def cos(arg: Union["Dual", Number]) -> "Dual":
    match arg:
        case Dual(arg.value, arg.d):
            return Dual(math.cos(arg.value), - math.sin(arg.value) * arg.d)
        case Number():
            return math.cos(arg)

def sin(arg: Union["Dual", Number]) -> "Dual":
    match arg:
        case Dual(arg.value, arg.d):
            return Dual(math.sin(arg.value), math.cos(arg.value) * arg.d)
        case Number():
            return math.sin(arg)

def log(arg: Union["Dual", Number]) -> "Dual":
    try:
        match arg:
            case Dual(arg.value, arg.d):
                return Dual(math.log(arg.value), arg.d / arg.value)
            case Number():
                return math.log(arg)
    except ValueError as error:
        print(f"The expression under logarithm is less than 0! {error=}")
        exit(-1)

f1 = lambda x: exp(x)
f2 = lambda x: cos(x)
f3 = lambda x: sin(x)
f4 = lambda x: log(x)

f1_diff = diff(f1)
f2_diff = diff(f2)
f3_diff = diff(f3)
f4_diff = diff(f4)
        
assert f1_diff(2.)[0] == math.exp(2.)
assert f2_diff(2.)[0] == - math.sin(2.)
assert f3_diff(2.)[0] == math.cos(2.)
assert f4_diff(2.)[0] == 1 / 2.


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [ ]:
from scipy.misc import derivative

def one_sides_diff_method(f: Callable[[float], float], x: float, h: float) -> float: # One-sided difference methods
    return (f(x + h) - f(x)) / h

f = lambda x: 5 * x * x + 2 * x + 2

print(one_sides_diff_method(f, 2., 1e-6))
print(derivative(f, 2., dx=1e-6))

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генераци случайных функций осуществляется в текстовом виде, с помощью встроенной функции [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
import random

vars = 1 # number of variables
variables = [f"x{i}" for i in range(0, vars)] # list of variables: x1, x2, ..., xn
const_max = 1e3 # max const value
functions = ["cos", "log", "exp", "sin", "abs", "const"] # base functions
operands = ["*", "/", "+", "-", "**"] # base operations
recoursion_depth = 0
max_recoursion_depth = 2
def generate_function():
    global recoursion_depth
    expression = ""
    ops = random.randint(0, 2)
    skip_left = False
    if recoursion_depth >= max_recoursion_depth:
        expression += random.choice(variables)
        return expression
    if ops == 0:
        # e.g. y = sin(log(x + 2))
        function = random.choice(functions + variables*2)
        if function == "const":
            expression += str(random.randint(1, const_max))
        elif function in variables:
            expression += function
        else:
            recoursion_depth += 1
            expression += function + "(" + generate_function() + ")"
    else:
        for _ in range(ops):
            op = random.choice(operands)
            left = random.choice(functions + variables)
            right = random.choice(functions + variables)
            if not skip_left:
                if left == "const":
                    expression += str(random.randint(1, const_max))
                elif left in variables:
                    expression += left
                else:
                    recoursion_depth += 1
                    expression += left + "(" + generate_function() + ")"
            expression += op
            if right == "const":
                expression += str(random.randint(1, const_max))
            elif right in variables:
                expression += right
            else:
                recoursion_depth += 1
                expression += right + "(" + generate_function() + ")"
            skip_left = True
    return expression


tests = 50
errors = 0

for i in range(tests):
    string_function = f"lambda {', '.join(variables)}: "
    string_function += generate_function()
    try:
        f = eval(string_function)
        f_diff = diff(f)
        x = random.uniform(-1000, 1000)
        if abs(f_diff(x)[0] - derivative(f, x, dx=1e-6)) > 1e-5:
            print(f"Error in function {string_function}, {x=}")
            print(f_diff(x)[0])
            print(derivative(f, x, dx=1e-6))
            errors += 1
    except OverflowError as e:
        print(f"{string_function=}, {x=}, {e=}")
        errors += 1
    except TypeError as e:
        print(f"{string_function=}, {x=}, {e=}")
        errors += 1
print(f"{errors} errors, {tests} tests")

In [ ]:
f = lambda x0: log(log(x0)**417/log(x0))+exp(x0)/x0
f_d = diff(f)
x = 131.2187717
print(f_d(x)[0])
print(derivative(f, x, dx=1e-6))
print(abs(f_d(x)[0] - derivative(f, x, dx=1e-6)) > 1)


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]